# Наивный байесовский классификатор
## Теория в двух словах

Вероятность - функция $P()$, определенная на событиях. Значение функции отражает наше представление о том, с какой вероятностью данное событие случается: 0 - не случается вовсе, 1 - случается всегда.

Сегодня нам потребуется два стандартных факта, о вероятностях.

### Условная вероятность и формула Байеса
Наши представления о том, что случится событие $A$, если мы знаем, что $B$ уже произошло традиционно вычисляются по формуле 

$p(A | B) = \frac{p(A B)}{p(B)}$. 

Здесь $p(A B)$ - вероятность того, что случится и А, и B.

Поменяем местами A и B в формуле, получим $p(B | A) = p(A B) / p(A)$, домножим обе части на $p(B)$, получится $p(AB) = p(B | A) p(A)$. Подставив это выражение в первую формулу, получаем

$ p(A | B) = \frac{p(B | A) p(A)}{p(B)} $

Эта формула называется формулой Байеса. Она позволяет нам скорректировать свое представление о том, вероятности $p(A)$ с учетом произошедшего события $B$.

### Независимости событий

В теории вероятности пара событий $A$ и $B$ называются независимыми, если выполнено соотношение

$p(AB) = p(A)p(B)$.

Определение продолжается очевидным образом: $n$ событий $A_1, ..., A_n$ называются независимыми, если выполнено

$p(A_1,...,A_n) = p(A_1)...p(A_n)$

### Комбинируем знания

Допустим, у нас есть обучающий объект $x$ из класса $y \in \{0, 1\}$, у обучающего объекта есть набор признаков $x_1, ..., x_n$.

Положим в формуле Байеса в качестве события $A$ событие "объект принадлежит классу $y$", а в качестве события $B$ событие "объект обладает признаками $x_1, ...,x_n$". Перепишем формулу в более удобных обозначениях:

$p(y | x_1, ..., x_n) = \frac{p(y) p(x_1, ..., x_n | y)}{p(x_1, ..., x_n)} $

Дальше мы можем сделать "наивное" предоложение о том, что все признакми объекта независимы между собой, зависят только от класса:

$p(x_1, ..., x_n | y) = \prod p(x_i | y) $

Это предположение нас приводит к формуле вероятности

$p(y | x_1, ..., x_n) = \frac{p(y) \prod p(x_i | y)}{p(x_1, ..., x_n)}$



# Пример
## Небольшой спам фильтр своими руками

Разберем работу наивного байесовского классификатора на примере. В качестве данных взяли набор sms-сообщений, нужно определить, является ли данное сообщение рекламным

In [ ]:
%matplotlib inline
import pandas as pd

data = pd.read_csv('SMSSpamCollection.txt',
                   sep = '\t',
                   names = ('label', 'text'))

Посмотрим на случайно взятое сообщение. Это спам?

In [ ]:
data.ix[3330].text

Работать с числами куда проще, нежели со строками. Каждое сообщение мы разобьем на слова, а каждому слову присвоим уникальный номер.

Ниже определена функция, разбивающая сообщения на слова (можно ли её улучшить?), а также словарь *mapping*, сопоставляющий каждому слову число.

In [ ]:
import re
from sklearn.feature_extraction.text import CountVectorizer

def split_message(message):
    return re.sub("[^a-zA-Z]", " ",  message.lower()).split()

mapping = CountVectorizer(analyzer=split_message).fit(data.text).vocabulary_

Как всегда, данные мы разобьем на обучающую и тестовую выборку.

In [ ]:
data_train = data[:4000]
data_test = data[4000:]

Вспомогательная функция, которая составляет список (с повторами) всех встречающихся слов в сообщениях типа *sms_type* в *data*. 

In [ ]:
def get_words(data, sms_type):
    return data[data.label == sms_type].text.apply(split_message).sum(axis = 0)

spam_words = get_words(data_train, 'spam')
ham_words = get_words(data_train, 'ham')

А в этой ячейки списки слов преобразуются в списки номеров слов.

In [ ]:
import numpy as np

words_to_numbers = lambda message: [mapping[word] for word in message]

spam_np = np.array(words_to_numbers(spam_words))
ham_np = np.array(words_to_numbers(ham_words))

Последние приготовления перед тем как написать классификатор. В нашем словаре всего $n$ слов, в обучающий выборке *N_spam* слов, встетившихся в рекламных сообщениях, *N_ham* слов, встретившихся в обычных сообщениях.

В массиве *spam_unique* хранятся номера слов, встретившихся в рекламных сообщениях, а массив *spam_counts* содержит данных о том, сколько раз каждое слово встретилось.

In [ ]:
n = len(mapping)
N_spam = len(spam_np)
N_ham = len(ham_np)

spam_unique, spam_counts = np.unique(spam_np, return_counts = True)
ham_unique, ham_counts = np.unique(ham_np, return_counts = True)

Например:

In [ ]:
print('Слово под номером %d встретилось в рекламных сообщениях %d раз.' % (spam_unique[3], spam_counts[3]))

Давайте вычислим вероятности, необходимые для работы классификатора.

Для каждого слова $x_i$ вероятность того, что оно принадлежит классу $y$ (спам или не спам) определяется формулой

$$ p(x_i | y) = \frac{N_{iy} + \alpha}{N_y + \alpha n} $$

Здесь $N_{iy}$ равно количеству раз, которое слово $i$ встретилось в сообщениях класса $y$ обучающей выборки, а $N_y$ равно количеству слов в классе $y$ в обучающей выборке.

А теперь можно написать и сам классификатор. Он вычисляет значение $p(y) \prod p(x_i | y)$ для двух значений классов, и выбирает тот класс $y$, для которого это значение больше.

В какой доле случаев класс предсказан правильно?

In [ ]:
(data_test.label == data_test.text.apply(classify)).mean()

Как часто спам оказывается помечен как обычное сообщение?

In [ ]:
(((data_test.label == 'spam') &
  (data_test.text.apply(classify) == 'ham')).mean() / 
     (data_test.label == 'spam').mean())

Как часто обычное сообщение оказывается помечено как спам?

In [ ]:
(((data_test.label == 'ham') & 
  (data_test.text.apply(classify) == 'spam')).mean() /
    (data_test.label == 'ham').mean())

Посмотрим на работу классификатора поближе:

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(20, 4))

ix = np.random.randint(0, data.shape[0])
example = data.text[ix]

print('Test message: %s \nclass:\t%s \nprediction: %s' % (example, data.label[ix], classify(example)))
words = split_message(example)[0:15]
numbers = words_to_numbers(words)

x = np.arange(len(words)) + 1
y_1 = p_spam[numbers]
y_2 = p_ham[numbers]

w_1 = np.prod(y_1) / (np.prod(y_1) + np.prod(y_2))
w_2 = np.prod(y_2) / (np.prod(y_1) + np.prod(y_2))

ax.plot(x, y_1, 'ro', lw = 2, label = ('spam, $w_{spam}$ = %f' % w_1))
ax.plot(x, y_2, 'bo', lw = 2, label = ('ham, $w_{ham}$ = %f' % w_2))

ax.set_xticks(x)
ax.set_xticklabels(words, fontsize=14)
ax.legend();